### [&ldquo;Low vaccination rates put some Oregon schools at high risk for measles&rdquo;](http://www.oregonlive.com/politics/index.ssf/2018/02/low_vaccination_rates_put_oreg.html)
#### Lynne Terry and Melissa Lewis, <em>The Oregonian</em>,  Feb. 11, 2018

In [1]:
import math

import numpy as np
import pandas as pd

<hr>

#### For this analysis, we're going to join two datasets:

1. A set of schools in Oregon categorized as `charter` or `traditional`, which we'll call `school_types_df`, derived from a table maintained by Mark Friesen.

1. A list of schools and the counts of students enrolled, as well as a count of students who have had a particular vaccine, `vax_df`, provided by [Oregon Health Authority](https://web.archive.org/web/20180210231248/http://www.oregon.gov/oha/PH/PREVENTIONWELLNESS/VACCINESIMMUNIZATION/GETTINGIMMUNIZED/Pages/SchRateShare.aspx).

In [2]:
school_types_df = pd.read_csv('data/schools_types.csv', dtype={'ID': str})

In [3]:
vax_df = pd.read_excel('data/SchK-12.xlsx', dtype={'ID': str})

#### We need to omit online schools in this analysis.
The following is a list of IDs of online schools provided by Lynne Terry.

In [4]:
online_schools = [
    '4', '488', '611', '1756', '1263', '1264',
    '869', '287', '3568', '649', '462', '194'
]

In [5]:
school_types_df.loc[school_types_df['ID'].isin(online_schools)]

,ID,Name,Type
2,4,Baker Web Academy,charter
131,194,Summit Learning Charter,charter
205,287,Oregon Virtual Academy,charter
332,462,Silvies River Charter School,charter
352,488,Crater Lake Charter Academy,charter
444,611,Eagle Ridge High School,charter
470,649,Paisley School District,charter
629,869,Oregon Connections Academy,traditional
904,1263,Me Web Academy High School,charter
905,1264,Me Web Academy Middle School,charter


In [6]:
# Reassign school_types_df to omit online schools
school_types_df = school_types_df.loc[
    ~school_types_df['ID'].isin(online_schools)]

In [7]:
school_types_df.head()

,ID,Name,Type
0,1,Baker High School,traditional
1,2,Baker Middle School,traditional
3,5,Brooklyn Elementary School,traditional
4,6,Burnt River School District,charter
5,7,Eagle Cap High School,traditional


In [8]:
vax_df.head()

,ID,County,SiteName,Grade,# Documentation Required (Adjusted Enrollment),# Vaccinated: DTaP/Tdap,# Vaccinated: DTaP/Tdap.1,# Vaccinated: Polio,% Vaccinated: Polio,# Vaccinated: Varicella,...,% Exemption: HepB,# Exemption: HepA,% Exemption: HepA,# Nonmedical Exemptions All Vaccines,% Nonmedical Exemptions All Vaccines,# No Record,% No Record,# w/ Medical Exemption(s),% w/ Medical Exemption(s),# Not Counted
0,1,Baker,BAKER HIGH SCHOOL,FULL SCHOOL GRADES K-12,464,450.0,0.969828,452.0,0.974138,453.0,...,0.019397,NaN,NaN,6.0,0.012931,0.0,0.000000,0.0,0.000000,0.0
1,2,Baker,BAKER MIDDLE SCHOOL,FULL SCHOOL GRADES K-12,202,191.0,0.945545,193.0,0.955446,194.0,...,0.034653,7.0,0.034653,6.0,0.029703,1.0,0.004950,1.0,0.004950,0.0
2,3,Baker,BAKER VALLEY SEVENTH-DAY ADVENTIST SCHOOL,FULL SCHOOL GRADES K-12,12,7.0,0.583333,7.0,0.583333,7.0,...,0.416667,NaN,NaN,5.0,0.416667,0.0,0.000000,0.0,0.000000,0.0
3,4,Baker,BAKER WEB ACADEMY,FULL SCHOOL GRADES K-12,996,771.0,0.774096,810.0,0.813253,829.0,...,0.167671,141.0,0.141566,150.0,0.150602,1.0,0.001004,2.0,0.002008,0.0
4,5,Baker,BROOKLYN ELEMENTARY SCHOOL,FULL SCHOOL GRADES K-12,457,430.0,0.940919,434.0,0.949672,440.0,...,0.032823,17.0,0.037199,12.0,0.026258,1.0,0.002188,0.0,0.000000,0.0


In [9]:
vax_df.rename(
    columns={'# Vaccinated:\nMeasles': '# Vaccinated',
             '# Documentation Required\n(Adjusted Enrollment)': 'Enrolled'},
    inplace=True)

In [10]:
vax_subset = vax_df[['ID', 'County', '# Vaccinated', 'Enrolled']]

In [11]:
school_vax = pd.merge(school_types_df, vax_subset, how='left', on='ID')

#### Now the data is ready for analysis.

In [12]:
school_vax.head()

,ID,Name,Type,County,# Vaccinated,Enrolled
0,1,Baker High School,traditional,Baker,452.0,464.0
1,2,Baker Middle School,traditional,Baker,194.0,202.0
2,5,Brooklyn Elementary School,traditional,Baker,436.0,457.0
3,6,Burnt River School District,charter,Baker,27.0,31.0
4,7,Eagle Cap High School,traditional,Baker,27.0,27.0


<hr>

# Vaccine effectiveness and &ldquo;herd immunity&rdquo;

&ldquo;Herd immunity&rdquo; has somewhat varying meanings, but: 

> &ldquo;A common implication of the term is that the risk of infection among susceptible individuals in a population is reduced by the presence and proximity of immune individuals (this is sometimes referred to as “indirect protection” or a “herd effect”).&rdquo;<sup><a href="#id1">1</a></sup>

Herd immunity depends on

1. the percent of that population vaccinated (the *vaccination rate*) and 
2. the *effectiveness* of the vaccine.



$$vaccine\ effectiveness \cdot vaccination\ rate = herd\ immunity$$

<hr>

### Effectiveness: Measles

The vaccination for measles is categorized in this dataset as `measles`, which the data dictionary does not describe any further except to say that this field is a count of students who have had two doses of a measles-containing vaccine.

We reached out to Stacy Matthews, the Oregon Public Health Division's immunization school law coordinator to request more comprehensive information on what vaccine this would have been, and she said that this was almost always MMR, but may have been a different vaccine in some cases.

The effectiveness rate of two doses of MMR for measles is 97%.

In [13]:
measles_1 = 0.93
measles_2 = 0.97

<hr>

## Herd Immunity Threshold (HIT)

| Infection | HIT (%) |<sup><a href="#id2">2</a></sup>
|-----------|---------|
| Measles   | 92-94   |
| Mumps     | 75-86   |

Herd Immunity Threshold refers to how much of population must be immune to have herd immunity, defined above. I'm not certain why it's expressed as a range, given that it's a threshold, but suspect its width is a function of uncertainty.

We consulted with [Dr. Paul Cieslak](https://web.archive.org/web/20180210215247/http://www.oregon.gov/oha/PH/PREVENTIONWELLNESS/VACCINESIMMUNIZATION/Pages/researchstaff.aspx#b9e7f508-53f7-4228-8551-5ebe9f0187eb), Medical Director of Communicable Diseases and Immunizations at Oregon Health Authority's Public Health Division, and decided to compare schools' immunity rates with both the minimum and the midpoint of that range.

In [14]:
measles_midpoint = round((0.92 + 0.94) / 2, 3)

measles_minimum = 0.92

I rounded the midpoint calculation results to two decimal places to avoid a specious inflation of precision.

<hr>

#### The following function uses the number of students enrolled, the number of students with vaccines, and that vaccine's effectiveness rating to calculate a school's immunity:

$$effectiveness \cdot percent\ vaccinated = herd\ immunity$$

In [15]:
school_vax['Percent Vaccinated'] = round(school_vax['# Vaccinated'] /
                                         school_vax['Enrolled'], 2)

In [16]:
school_vax.head()

,ID,Name,Type,County,# Vaccinated,Enrolled,Percent Vaccinated
0,1,Baker High School,traditional,Baker,452.0,464.0,0.97
1,2,Baker Middle School,traditional,Baker,194.0,202.0,0.96
2,5,Brooklyn Elementary School,traditional,Baker,436.0,457.0,0.95
3,6,Burnt River School District,charter,Baker,27.0,31.0,0.87
4,7,Eagle Cap High School,traditional,Baker,27.0,27.0,1.00


In [17]:
school_vax['Percent Immune'] = round(
    school_vax['Percent Vaccinated'] * measles_2, 2)

In [18]:
def measles_threshold_calc(row):
    if row['Percent Immune'] >= measles_midpoint:
        threshold = 'midpoint'
    elif row['Percent Immune'] >= measles_minimum:
        threshold = 'minimum'
    else: threshold = 'none'
    return threshold

In [19]:
school_vax['Measles HIT Met'] = school_vax.apply(
    lambda x: measles_threshold_calc(x), axis=1)

In [20]:
school_vax.sort_values(by='Percent Immune').iloc[0:10]

,ID,Name,Type,County,# Vaccinated,Enrolled,Percent Vaccinated,Percent Immune,Measles HIT Met
432,604,Woodland Charter School,charter,Josephine,60.0,158.0,0.38,0.37,none
569,802,Village School,charter,Lane,96.0,211.0,0.45,0.44,none
967,1407,Dallas Community School,charter,Polk,89.0,156.0,0.57,0.55,none
364,509,John Muir Elementary,traditional,Jackson,74.0,123.0,0.60,0.58,none
369,516,Madrone Trail Public Charter School,charter,Jackson,146.0,233.0,0.63,0.61,none
940,1359,Southwest Charter School,charter,Multnomah,127.0,190.0,0.67,0.65,none
767,1066,The Community Roots School,charter,Marion,70.0,102.0,0.69,0.67,none
7,12,Keating Elementary School,traditional,Baker,18.0,26.0,0.69,0.67,none
912,1321,Portland Village School,charter,Multnomah,274.0,394.0,0.70,0.68,none
174,251,South Columbia Family School,charter,Columbia,53.0,75.0,0.71,0.69,none


In [21]:
school_vax.to_csv('schools_and_immunity.csv')

# Notes

**<a id="id1">[1]</a>**  Paul Fine, Ken Eames, David L. Heymann; “Herd Immunity”: A Rough Guide, *Clinical Infectious Diseases*, Volume 52, Issue 7, 1 April 2011, Pages 911–916, https://doi.org/10.1093/cid/cir007


**<a id="id2">[2]</a>**  Stanley A. Plotkin, Walter Orenstein, Paul A. Offit, and Kathryn M. Edwards, *Vaccines* (Elsevier Health Sciences, 2017), https://books.google.com/books?id=yUijDgAAQBAJ&lpg=PA1515&pg=PA1515